In [102]:
%load_ext autoreload
%autoreload 2

from itertools import combinations
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.spatial import distance_matrix

from generate.symmetry_config import *
from generate.generate_color_patterns import *

from utils.compressions import compress_for_color

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
df = pd.read_csv('data/color_patterns/color_pattern_dataset0.csv')
df.colors = df.colors.map(eval)
# TODO: ulozit do listu vsechny dfs, a pak je v metrikach prochazet

In [99]:
def indices_to_position(indices):
    pos_array = np.array(pos_list)
    return pos_array[indices]

# Continuous Metrics

## Euclidean

In [14]:
def calc_euclidean(vertices):
    vertices = indices_to_position(vertices)
    distances = distance_matrix(vertices, vertices)
    distances = np.sort(distances.sum(axis=1))
    distances = np.rint(distances*10e4).astype(int)
    return distances

In [15]:
compress_for_color(df, calc_euclidean)

 93%|██████████████████████████████████████████████████████████████████████▊     | 22084/23697 [06:32<00:28, 56.33it/s]

ERROR: Same hashes (6025586652036973081) for symmetry class 22084 and 22000


# Discrete Metrics

## Manhattan

In [16]:
def calc_manhattan(vertices):
    vertices = indices_to_position(vertices)
    distances = distance_matrix(vertices, vertices, p=1)
    distances = np.sort(distances.sum(axis=1))
    distances = np.rint(distances*10e4).astype(int)
    return distances

In [17]:
compress_for_color(df, calc_manhattan)

  0%|                                                                                | 5/23697 [00:00<05:41, 69.43it/s]

ERROR: Same hashes (-7184283474170153106) for symmetry class 5 and 1


## Chebyshev (max norm)

In [18]:
def calc_chebyshev(vertices):
    vertices = indices_to_position(vertices)
    distances = distance_matrix(vertices, vertices, p=np.inf)
    distances = np.sort(distances.sum(axis=1))
    distances = np.rint(distances*10e4).astype(int)
    return distances

In [19]:
compress_for_color(df, calc_chebyshev)

  0%|                                                                                | 2/23697 [00:00<08:34, 46.04it/s]

ERROR: Same hashes (-508231428013540880) for symmetry class 2 and 1


## Surface distance (cube) - discrete

In [110]:
def calc_surface_dist(vertices):
    positions_on_cross_arrs = np.array(positions_on_cross_list)
    cross_pos_acc = []
    for k, vertex in enumerate(vertices):
        cross_pos_acc.append([])
        for positions_on_cross_arr in positions_on_cross_arrs:
            positions_on_cross = np.transpose((positions_on_cross_arr == vertex).nonzero()).squeeze()
#             print(positions_on_cross_arr == vertex)
            cross_pos_acc[k].append(positions_on_cross)
    cross_pos_acc = np.array(cross_pos_acc)
#     print(cross_pos_acc)
    
    distances = np.zeros((len(vertices), len(vertices)))
    for i, pos1 in enumerate(cross_pos_acc):
        for j, pos2 in enumerate(cross_pos_acc):
            if i == j: 
                continue
#             print(pos1.shape, pos2.shape)
            dist = np.sum(np.abs(pos1 - pos2), axis=1)
            dist = np.min(dist)
#             print(vertices[i], vertices[j], dist)
            distances[i, j] = dist
    
    distances = np.sort(distances.sum(axis=1))
    distances = np.rint(distances*10e4).astype(int)
    return distances

In [111]:
print(calc_surface_dist(df.iloc[48]['colors']))
print('-----------------------------------------------')
print(calc_surface_dist(df.iloc[49]['colors']))

[1600000 1700000 1900000 1900000 2000000 2200000 2200000 2400000 4500000]
-----------------------------------------------
[1600000 1700000 1900000 1900000 2000000 2200000 2200000 2400000 4500000]


In [112]:
compress_for_color(df, calc_surface_dist)

  0%|                                                                               | 10/23697 [00:01<41:37,  9.48it/s]

ERROR: Same hashes (6759901364409641569) for symmetry class 10 and 7
